In [ ]:
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.spatial import cKDTree
from scipy.stats import binned_statistic
from scipy.interpolate import interp1d
from tqdm import tqdm

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

from totoro.config import galcen_frame, elem_names
from totoro.data import datasets, elem_names
from totoro.potentials import potentials, galpy_potentials
from totoro.objective import TorusImagingObjective

In [ ]:
data_name = 'galah-rgb-loalpha'
d = datasets[data_name]

In [ ]:
elem_name = elem_names[data_name][0]

In [ ]:
tree_K = 64

In [ ]:
x0 = np.array([1.1, 20.8, 7.78])  # HACK: init from fiducial

In [ ]:
rnd = np.random.default_rng(42)
idx = rnd.choice(len(d), len(d), replace=True)
obj = TorusImagingObjective(d[idx], elem_name,
                            tree_K=tree_K)
i = 0

In [ ]:
atm = obj.get_atm(*x0)
atm.get_coeffs_for_elem(elem_name)

In [ ]:
res = None
try:
    res = obj.minimize(x0=x0, method="nelder-mead",
                       options=dict(maxiter=1024))
    print(f"{i} finished optimizing: {res}")
except Exception as e:
    print(f"{i} failed: {str(e)}")

In [ ]:
if res is None or not res.success:
    xopt = np.nan * np.array(x0)
else:
    xopt = res.x

xopt = {
    'mdisk_f': [xopt[0]],
    'zsun': [xopt[1]],
    'vzsun': [xopt[2]]
}